In [31]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings


In [32]:
CHROMA_PATH = "chroma"
data_path = "data/monopoly.pdf"

In [33]:
def load_documents():
    loader = PyPDFLoader(data_path)
    documents = loader.load()
    return documents

documents = load_documents()

In [34]:
def split_text(documents: list):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 500,
        length_function = len,
        add_start_index = True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} into {len(chunks)} chunks.")
    return chunks

In [35]:
chunks = split_text(documents)

Split 8 into 29 chunks.


In [36]:
print(chunks[0].page_content)
print(chunks[0].metadata)

MONOPOLY 
Property Trading Game from Parker Brothers" 
AGES 8+ 
2 to 8 Players 
Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance 
and Community Chest cards, Title Deed cards, play money and a Banker's tray. 
Now there's a faster way to play MONOPOLY. Choose to play by 
the classic rules for buying, renting and selling properties or use the 
Speed Die to get into the action faster. If you've never played the classic 
MONOPOLY game, refer to the Classic Rules beginning on the next page. 
If you already know how to play and want to use the Speed Die, just 
read the section below for the additional Speed Die rules. 
SPEED DIE RULES 
Learnins how to Play with the S~eed Die IS as 
/ 
fast as playing with i't. 
1. When starting the game, hand out an extra $1,000 to each player 
(two $5005 should work). The game moves fast and you'll need 
the extra cash to buy and build. 
2. Do not use the Speed Die until you've landed on or passed over
{'producer': 'Adobe Acrobat 7.0 Paper C

In [38]:
db = Chroma.from_documents(
    chunks, OpenAIEmbeddings(), persist_directory = CHROMA_PATH
) 

/var/folders/4k/zr7r4x5s7szf89g_d1c944m80000gn/T/ipykernel_62085/650719165.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  chunks, OpenAIEmbeddings(), persist_directory = CHROMA_PATH


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error